# Introduction to QAX

Author: R.Maekura (ryomaekura@g.ecc.u-tokyo.ac.jp)

## Import package

In [2]:
import jax
import jax.numpy as jnp

import sys
sys.path.append("/home/users/u0001529/ondemand/qax-project")

%load_ext autoreload
%autoreload 2
from qax import state
from qax import operator as op
from qax.utils import linalg
from qax.utils import device

## Quantum states

The matrix representations of commonly used quantum states can be accessed via `qax.state`. \
In addition, basic information such as the shape and type of an operator can be displayed using `qax.utils.display.info`

### state vector

In [7]:
# initial n-qubits state
n = 2
qubit_state = state.n_qubit(n)
qubit_state.info()

Shape: (4,)
Dtype: complex64
Norm: 1.000000


<IPython.core.display.Math object>

In [12]:
# vaccum state
sys_dim = 10
vac_state = state.vacuum(sys_dim)
vac_state.info()

Shape: (10,)
Dtype: complex64
Norm: 1.000000


<IPython.core.display.Math object>

In [9]:
# fock state
sys_dim = 10
n = 5
fock_state = state.fock(n, sys_dim)
fock_state.info()

Shape: (10,)
Dtype: complex64
Norm: 1.000000


<IPython.core.display.Math object>

### density matrix

In QAX, the function `qax.utils.linalg.vec2dm()` is used to convert the state vector into a density matrix.

In [10]:
# density matrix of initial n-qubits state
qubit_dm = linalg.vec2dm(qubit_state)
qubit_dm.info()

Shape: (4, 4)
Dtype: complex64
Hermitian: True


<IPython.core.display.Math object>

In [11]:
# density matrix of fock state
fock_dm = linalg.vec2dm(fock_state)
fock_dm.info()

Shape: (10, 10)
Dtype: complex64
Hermitian: True


<IPython.core.display.Math object>

## Quantum operators

### 2-level system

In [3]:
# pauli operators
## pauli x
pauli_x = op.pauli_x()
pauli_x.info()

Shape: (2, 2)
Dtype: complex64
Is Hermitian: True


<IPython.core.display.Math object>

In [4]:
## pauli y
pauli_y = op.pauli_y()
pauli_y.info()

Shape: (2, 2)
Dtype: complex64
Is Hermitian: True


<IPython.core.display.Math object>

In [5]:
## pauli z
pauli_z = op.pauli_z()
pauli_z.info()

Shape: (2, 2)
Dtype: complex64
Is Hermitian: True


<IPython.core.display.Math object>

### finite-level system

In [3]:
# annihilation operator
sys_dim = 5
a = op.annihilation(sys_dim)
a.info()

Shape: (5, 5)
Dtype: complex64
Hermitian: False


<IPython.core.display.Math object>

In [4]:
# creation operator
a_dag = op.annihilation(sys_dim).dagger()
a_dag.info()

Shape: (5, 5)
Dtype: complex64
Hermitian: False


<IPython.core.display.Math object>

In [6]:
# position operator
x = op.position(sys_dim)
x.info()

Shape: (5, 5)
Dtype: complex64
Hermitian: True


<IPython.core.display.Math object>

In [7]:
# momentum operator
p = op.momentum(sys_dim)
p.info()

Shape: (5, 5)
Dtype: complex64
Hermitian: True


<IPython.core.display.Math object>

## Time evolution

As a demonstration, the time evolution of the system is calculated using the following Jaynes–Cummings Hamiltonian.
$$
\begin{align*}
H &= \omega_{\rm c} \hat{a}^\dagger \hat{a} - \frac{1}{2} \omega_{\rm a} \sigma_{\rm z} + g \left( \hat{a} \sigma_+ + \hat{a}^\dagger \sigma_- \right)
\end{align*}
$$

In [ ]:
# Jaynes-Cumming Hamiltonian
## physical parameter settings
omega_c = 1.0  
omega_a = 1.0  
g = 0.1

## operator settings |atom_state, cavity_state>
cavity_dim = 100
atom_dim = 2

pauli_z = linalg.kron_prod(op.pauli_z(), op.identity(cavity_dim))
a = linalg.kron_prod(op.identity(atom_dim), op.creation(cavity_dim))
a_dag = linalg.kron_prod(op.identity(atom_dim), op.annihilation(cavity_dim))

H = omega_c*a@a_dag - 0.5*omega_a*pauli_z + g*(jnp.dot()+jnp.dot())

### time-dependent Schrödinger equation

$$
\begin{align*}
ih \frac{\partial}{\partial t} \ket{\Psi} &= \hat{H} \ket{\Psi}
\end{align*}
$$

In [16]:
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

# あなたが作成したライブラリのモジュールをインポート
# (パッケージ名が 'qax' であると仮定)
from qax import state, operator
from qax.dynamics.solver.schrodinger import SchrodingerEquation
import diffrax # diffraxを直接使うため

# --- 1. 物理パラメータを定義 (単位: 2π GHz) ---
#    (例: 5.0 は 5.0 * 2π GHz を意味する)
delta_phys = 5.0       # 量子ビットのエネルギーギャップ
omega_rabi_phys = 2.0  # 駆動の強さ (ラビ周波数)
drive_freq_phys = 5.0  # 外部から加える駆動の周波数 (共鳴状態)


# --- 2. 無次元化の実行 ---
# 基準となる周波数を設定 (通常、系の主要なエネルギースケールを選ぶ)
omega_c = delta_phys

# 無次元パラメータを計算
# これらはすべて1前後のオーダーの数値になる
delta_dimless = delta_phys / omega_c      # -> 1.0
omega_rabi_dimless = omega_rabi_phys / omega_c # -> 0.4
drive_freq_dimless = drive_freq_phys / omega_c # -> 1.0


# --- 3. 無次元化されたハミルトニアンを定義 ---
# H'(τ) = (Δ/ωc)/2 * σ_z + (Ω/ωc) * cos((ω/ωc) * τ) * σ_x
H0_op = (delta_dimless / 2) * operator.pauli_z()
H_drive_op = operator.pauli_x()

# 時間t (無次元化された時間τ) を引数に取り、ハミルトニアン(Operator)を返す関数
def hamiltonian_t_dimless(t: float) -> Operator:
    drive_term = omega_rabi_dimless * jnp.cos(drive_freq_dimless * t) * H_drive_op
    return H0_op + drive_term


# --- 4. 初期状態とシミュレーション時間を設定 ---
# 初期状態 |ψ(0)⟩ = |0⟩ (基底状態)
psi0 = state.fock(0, dim=2)

# シミュレーション時間も無次元化する
# 例えば、物理時間で 2.5 ns までシミュレーションしたい場合:
# t_phys_end = 2.5 (ns)
# τ_end = omega_c * t_phys_end = 5.0 * 2.5 = 12.5
t_span_dimless = (0.0, 12.5) 

# 結果を保存する時間点を設定
ts_to_save = jnp.linspace(t_span_dimless[0], t_span_dimless[1], 201)
saveat = diffrax.SaveAt(ts=ts_to_save)


# --- 5. ソルバーを準備して実行 ---
# 無次元化されたハミルトニアンを使ってソルバーをインスタンス化
solver = SchrodingerEquation(
    hamiltonian=hamiltonian_t_dimless,
    initial_state=psi0,
    t_span=t_span_dimless
    # hbar=1 はデフォルトなので省略
)

# 時間発展を実行
solver.run(saveat=saveat)


# --- 6. 結果の分析とプロット ---
print("シミュレーションが完了しました。結果をプロットします。")

# σ_zの期待値を計算
sigma_z = operator.pauli_z()
times_dimless, exp_vals_z = solver.expectation_value(sigma_z)

# プロット
fig, ax1 = plt.subplots(figsize=(10, 6))

# 無次元の時間でプロット
ax1.plot(np.asarray(times_dimless), np.asarray(exp_vals_z), color='mediumblue')
ax1.set_xlabel("Dimensionless Time  (τ = ω_c * t)", fontsize=14)
ax1.set_ylabel("Expectation Value <σ_z>", fontsize=14)
ax1.set_title("Rabi Oscillation (Dimensionless)", fontsize=16)
ax1.grid(True)
ax1.set_ylim(-1.1, 1.1)

# --- (オプション) 物理的な時間スケールを上部x軸に追加 ---
ax2 = ax1.twiny()
# τ を t_phys に変換 (t = τ / ω_c)
min_phys_time = times_dimless.min() / omega_c
max_phys_time = times_dimless.max() / omega_c
ax2.set_xlim(min_phys_time, max_phys_time)
ax2.set_xlabel("Physical Time (ns)", fontsize=14)

fig.tight_layout()
plt.show()

2025-07-10 07:02:14.249193: W external/xla/xla/service/gpu/ir_emitter_unnested.cc:1180] Unable to parse backend config for custom call: Could not convert JSON string to proto: : Root element must be a message.
Fall back to parse the raw backend config str.
2025-07-10 07:02:14.249597: W external/xla/xla/service/gpu/ir_emitter_unnested.cc:1180] Unable to parse backend config for custom call: Could not convert JSON string to proto: : Root element must be a message.
Fall back to parse the raw backend config str.


シミュレーションが完了しました。結果をプロットします。


ValueError: Operator must be initialized with a square 2D array, but got shape (1, 2)

### master equation

In [7]:
from qax import operator, state
from qax.utils import device
import jax.numpy as jnp

print("--- CPUでデバッグ実行 ---")

# すべてのオブジェクトをCPUに配置
cpu = device.select_device('cpu')
H_cpu = device.to(operator.pauli_x(), cpu)
psi0_cpu = device.to(state.fock(0, 2), cpu)

# 計算を実行
result_cpu = H_cpu @ psi0_cpu

print(f"演算子のデバイス: {list(H_cpu.data.devices())[0]}")
print(f"状態ベクトルのデバイス: {list(psi0_cpu.data.devices())[0]}")
print(f"結果のデバイス: {list(result_cpu.data.devices())[0]}")

--- CPUでデバッグ実行 ---
演算子のデバイス: TFRT_CPU_0
状態ベクトルのデバイス: TFRT_CPU_0
結果のデバイス: TFRT_CPU_0


In [9]:
from qax.utils import device
import jax

print("--- GPUがあれば使い、なければCPUにフォールバック ---")
try:
    # プライマリGPUの取得を試みる
    target_device = device.select_gpu()
    print("GPUを検出しました。計算はGPUで行います。")
except ValueError:
    # GPUが見つからなければCPUを使う
    target_device = device.select_cpu()
    print("GPUが見つかりません。計算はCPUで行います。")

# データを作成し、選択したデバイスに配置
my_array = jax.random.normal(jax.random.PRNGKey(0), (100, 100))
my_array_on_device = device.to(my_array, target_device)

print(f"最終的な配列のデバイス: {list(my_array_on_device.devices())[0]}")

--- GPUがあれば使い、なければCPUにフォールバック ---
GPUを検出しました。計算はGPUで行います。
最終的な配列のデバイス: cuda:0


In [10]:
from qax.utils import device
import jax

print("--- マルチGPU並列計算の準備 ---")
try:
    # 利用可能なすべてのGPUデバイスを取得
    gpu_list = device.list_gpus()
    print(f"{len(gpu_list)}個のGPUを検出しました: {gpu_list}")

    # このリストを `jax.pmap` に渡して並列化する
    # @jax.pmap(devices=gpu_list)
    # def parallel_function(data_chunk):
    #     # ... 各GPUでの計算 ...
    #     return result_chunk

except ValueError as e:
    print(e)

--- マルチGPU並列計算の準備 ---
1個のGPUを検出しました: [CudaDevice(id=0)]


In [12]:
from qax.core import Operator, StateVector
from qax.utils import device

class SchrodingerEquation:
    def __init__(self, hamiltonian: Operator, initial_state: StateVector, device_str: str | None = None):
        """
        ソルバーを初期化し、オブジェクトをターゲットデバイスに移動させる
        """
        # device.pyのヘルパーでデバイスオブジェクトを取得
        self.device = device.select_device(device_str)
        print(f"ソルバーはデバイス {self.device} 上で実行されます。")

        # device.to() を使ってオブジェクトを移動
        self.hamiltonian = device.to(hamiltonian, self.device)
        self.initial_state = device.to(initial_state, self.device)
        
# --- クラスの使い方 ---
from qax import operator, state

H = operator.pauli_z()
psi0 = state.fock(0, 2)

# ソルバーをインスタンス化する際に、'gpu'を指定
try:
    solver_gpu = SchrodingerEquation(H, psi0, device_str='gpu')
except ValueError as e:
    print(e)

ソルバーはデバイス cuda:0 上で実行されます。
